# 비행기 연착 추측 분류
- 분류 분석

In [ ]:
# 1. 비행정보 + 출발 공항정보
# 2. 비행정보 + 출발 공항정보 + 도착 공항정보
# 3. 컬럼 선택
# 4. 결측치 포함한 행 삭제
# 5. 컬럼 타입 변경
# 6. 명목형 변수 라벨 인코딩

## 패키지 로딩

In [109]:
import numpy as np
import pandas as pd

## 데이터 로드

In [110]:
airport_origin = pd.read_csv('Airport Codes Dataset.csv')
airport_origin.shape # (365,4)
airport_origin.head()

,airport_id,city,state,name
0,10165,Adak Island,AK,Adak
1,10299,Anchorage,AK,Ted Stevens Anchorage International
2,10304,Aniak,AK,Aniak Airport
3,10754,Barrow,AK,Wiley Post/Will Rogers Memorial
4,10551,Bethel,AK,Bethel Airport


In [111]:
flight_origin = pd.read_csv('Flight on-time performance.csv')
flight_origin.shape # (504397,18)
flight_origin.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepTimeBlk,DepDelay,DepDel15,CRSArrTime,ArrTimeBlk,ArrDelay,ArrDel15,Cancelled,Diverted
0,2011,4,10,6,4,WN,13495,12191,1435,1400-1459,2.0,0.0,1550,1500-1559,-6.0,0.0,0,0
1,2011,4,10,6,4,WN,13495,12191,1330,1300-1359,-4.0,0.0,1445,1400-1459,-12.0,0.0,0,0
2,2011,4,10,6,4,WN,13495,12191,1030,1000-1059,-2.0,0.0,1145,1100-1159,-14.0,0.0,0,0
3,2011,4,10,6,4,WN,13495,12889,1900,1900-1959,0.0,0.0,2055,2000-2059,-6.0,0.0,0,0
4,2011,4,10,6,4,WN,13495,12889,1340,1300-1359,-1.0,0.0,1530,1500-1559,2.0,0.0,0,0


In [115]:
# 카피본 떠놓고 시작하기
airport1 = airport_origin.copy()
airport1.info()

airport2 = airport_origin.copy()
airport2.info()

flight = flight_origin.copy()
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   airport_id  365 non-null    int64 
 1   city        365 non-null    object
 2   state       365 non-null    object
 3   name        365 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   airport_id  365 non-null    int64 
 1   city        365 non-null    object
 2   state       365 non-null    object
 3   name        365 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504397 entries, 0 to 504396
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   

## 데이터 전처리 1

In [116]:
# 1-1. 결측치 포함한 행 삭제
airport1.isna().sum()
flight.isna().sum()
# flight[flight['DepDelay'].isna()]

Year                  0
Quarter               0
Month                 0
DayofMonth            0
DayOfWeek             0
Carrier               0
OriginAirportID       0
DestAirportID         0
CRSDepTime            0
DepTimeBlk            0
DepDelay           3697
DepDel15           3697
CRSArrTime            0
ArrTimeBlk            0
ArrDelay           4717
ArrDel15           4717
Cancelled             0
Diverted              0
dtype: int64

In [117]:
flight.dropna(inplace=True)
# df_flight_cleared = flight.dropna()
flight.isna().sum()

Year               0
Quarter            0
Month              0
DayofMonth         0
DayOfWeek          0
Carrier            0
OriginAirportID    0
DestAirportID      0
CRSDepTime         0
DepTimeBlk         0
DepDelay           0
DepDel15           0
CRSArrTime         0
ArrTimeBlk         0
ArrDelay           0
ArrDel15           0
Cancelled          0
Diverted           0
dtype: int64

## 데이터 전처리 2

In [118]:
# 2-1. 출발 공항정보 만들기
airport1.columns = ['airport_id', 'Ori-city', 'Ori-state', 'Ori-airport']
# airport1 = airport1.rename(columns={'city':'Ori-city', 'state':'Ori-state', 'name':'Ori-airport'})
airport1.columns
# airport1.head()

Index(['airport_id', 'Ori-city', 'Ori-state', 'Ori-airport'], dtype='object')

In [119]:
# 2-2. 비행정보 + 출발 공항정보 병합하기
merged_df1 = pd.merge(flight, airport1, left_on='OriginAirportID', right_on='airport_id', how='left')
# 공통키만 가지고 조인하므로 inner join
# left : 왼쪽의 flight를 기준으로 left inner join
display(merged_df1.head())
merged_df1.shape # (499680, 22)

,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepTimeBlk,...,CRSArrTime,ArrTimeBlk,ArrDelay,ArrDel15,Cancelled,Diverted,airport_id,Ori-city,Ori-state,Ori-airport
0,2011,4,10,6,4,WN,13495,12191,1435,1400-1459,...,1550,1500-1559,-6.0,0.0,0,0,13495,New Orleans,LA,Louis Armstrong New Orleans International
1,2011,4,10,6,4,WN,13495,12191,1330,1300-1359,...,1445,1400-1459,-12.0,0.0,0,0,13495,New Orleans,LA,Louis Armstrong New Orleans International
2,2011,4,10,6,4,WN,13495,12191,1030,1000-1059,...,1145,1100-1159,-14.0,0.0,0,0,13495,New Orleans,LA,Louis Armstrong New Orleans International
3,2011,4,10,6,4,WN,13495,12889,1900,1900-1959,...,2055,2000-2059,-6.0,0.0,0,0,13495,New Orleans,LA,Louis Armstrong New Orleans International
4,2011,4,10,6,4,WN,13495,12889,1340,1300-1359,...,1530,1500-1559,2.0,0.0,0,0,13495,New Orleans,LA,Louis Armstrong New Orleans International


(499680, 22)

In [120]:
# 2-3. airport_id 컬럼 삭제
merged_df1.drop('airport_id', axis=1, inplace=True)
merged_df1.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier',
       'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'DepTimeBlk',
       'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrTimeBlk', 'ArrDelay',
       'ArrDel15', 'Cancelled', 'Diverted', 'Ori-city', 'Ori-state',
       'Ori-airport'],
      dtype='object')

## 데이터 전처리 3

In [121]:
# 3-1. 도착 공항정보 만들기
airport2.columns = ['airport_id', 'Dest-city', 'Dest-state', 'Dest-airport']
# airport2 = airport2.rename(columns={'city':'Dest-city', 'state':'Dest-state', 'name':'Dest-airport'})
airport2.columns
# airport2.head()

Index(['airport_id', 'Dest-city', 'Dest-state', 'Dest-airport'], dtype='object')

In [122]:
# 3-2. 비행정보 + 출발 공항정보 + 도착 공항정보 병합하기
merged_df2 = pd.merge(merged_df1, airport2, left_on='OriginAirportID', right_on='airport_id', how='left')
display(merged_df2.head())
merged_df2.shape # (499680, 25)

,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepTimeBlk,...,ArrDel15,Cancelled,Diverted,Ori-city,Ori-state,Ori-airport,airport_id,Dest-city,Dest-state,Dest-airport
0,2011,4,10,6,4,WN,13495,12191,1435,1400-1459,...,0.0,0,0,New Orleans,LA,Louis Armstrong New Orleans International,13495,New Orleans,LA,Louis Armstrong New Orleans International
1,2011,4,10,6,4,WN,13495,12191,1330,1300-1359,...,0.0,0,0,New Orleans,LA,Louis Armstrong New Orleans International,13495,New Orleans,LA,Louis Armstrong New Orleans International
2,2011,4,10,6,4,WN,13495,12191,1030,1000-1059,...,0.0,0,0,New Orleans,LA,Louis Armstrong New Orleans International,13495,New Orleans,LA,Louis Armstrong New Orleans International
3,2011,4,10,6,4,WN,13495,12889,1900,1900-1959,...,0.0,0,0,New Orleans,LA,Louis Armstrong New Orleans International,13495,New Orleans,LA,Louis Armstrong New Orleans International
4,2011,4,10,6,4,WN,13495,12889,1340,1300-1359,...,0.0,0,0,New Orleans,LA,Louis Armstrong New Orleans International,13495,New Orleans,LA,Louis Armstrong New Orleans International


(499680, 25)

In [123]:
# 3-3. airport_id 컬럼 삭제
merged_df2.drop('airport_id', axis=1, inplace=True)
merged_df2.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier',
       'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'DepTimeBlk',
       'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrTimeBlk', 'ArrDelay',
       'ArrDel15', 'Cancelled', 'Diverted', 'Ori-city', 'Ori-state',
       'Ori-airport', 'Dest-city', 'Dest-state', 'Dest-airport'],
      dtype='object')

## 데이터 전처리 4

In [124]:
# 4. 14개 컬럼 선택
df_flight_info = merged_df2[['DayOfWeek', 'Carrier', 'DepTimeBlk', 'DepDelay', 'DepDel15', 'ArrTimeBlk', 'ArrDelay', 'ArrDel15', 'Ori-city', 'Ori-state', 'Ori-airport', 'Dest-city', 'Dest-state', 'Dest-airport']]
df_flight_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499680 entries, 0 to 499679
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DayOfWeek     499680 non-null  int64  
 1   Carrier       499680 non-null  object 
 2   DepTimeBlk    499680 non-null  object 
 3   DepDelay      499680 non-null  float64
 4   DepDel15      499680 non-null  float64
 5   ArrTimeBlk    499680 non-null  object 
 6   ArrDelay      499680 non-null  float64
 7   ArrDel15      499680 non-null  float64
 8   Ori-city      499680 non-null  object 
 9   Ori-state     499680 non-null  object 
 10  Ori-airport   499680 non-null  object 
 11  Dest-city     499680 non-null  object 
 12  Dest-state    499680 non-null  object 
 13  Dest-airport  499680 non-null  object 
dtypes: float64(4), int64(1), object(9)
memory usage: 57.2+ MB


## 데이터 전처리 5

In [134]:
for col in [col for col in df_flight_info.columns if df_flight_info[col].dtype == 'object']: # list comprehension : 데이터 타입이 object인 column들만!
    print(f'컬럼명 :{col:15s}, 유니크값 : {len(df_flight_info[col].unique())}')
    # (-) 왼쪽 정렬로 (15s)15자리의 string 문자 간격으로
    # unique() : 각 컬럼별 유니크 값을 numpy의 다차원 배열로 갖고 온다.

컬럼명 :Carrier        , 유니크값 : 16
컬럼명 :DepTimeBlk     , 유니크값 : 19
컬럼명 :ArrTimeBlk     , 유니크값 : 19
컬럼명 :Ori-city       , 유니크값 : 268
컬럼명 :Ori-state      , 유니크값 : 53
컬럼명 :Ori-airport    , 유니크값 : 279
컬럼명 :Dest-city      , 유니크값 : 268
컬럼명 :Dest-state     , 유니크값 : 53
컬럼명 :Dest-airport   , 유니크값 : 279


In [135]:
# 5. 명목형 변수 : 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

for col in [col for col in df_flight_info.columns if df_flight_info[col].dtype == 'object']:
    df_flight_info[col]=encoder.fit_transform(df_flight_info[col])

C:\Users\user\AppData\Local\Temp\ipykernel_13304\2842582857.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info[col]=encoder.fit_transform(df_flight_info[col])


In [136]:
df_flight_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499680 entries, 0 to 499679
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DayOfWeek     499680 non-null  int64  
 1   Carrier       499680 non-null  int32  
 2   DepTimeBlk    499680 non-null  int32  
 3   DepDelay      499680 non-null  float64
 4   DepDel15      499680 non-null  float64
 5   ArrTimeBlk    499680 non-null  int32  
 6   ArrDelay      499680 non-null  float64
 7   ArrDel15      499680 non-null  float64
 8   Ori-city      499680 non-null  int32  
 9   Ori-state     499680 non-null  int32  
 10  Ori-airport   499680 non-null  int32  
 11  Dest-city     499680 non-null  int32  
 12  Dest-state    499680 non-null  int32  
 13  Dest-airport  499680 non-null  int32  
dtypes: float64(4), int32(9), int64(1)
memory usage: 40.0 MB


## 데이터 준비

In [137]:
x = df_flight_info.drop('ArrDel15', axis=1)
# x = df_flight_info.iloc[:, 1:]
y = df_flight_info['ArrDel15']
print(y.value_counts()) # y의 유니크값 분포 보기

0.0    431461
1.0     68219
Name: ArrDel15, dtype: int64


In [ ]:
# df3['ArrDel15'] = df3['ArrDel15'].astype(int)
# df3['ArrDel15'].info()

## 학습/평가용 데이터 분할

In [138]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.8,
                                                    stratify=y) # 랜덤 X
# stratify=y : 두 클래스 0와 1를 train/test에 동일한 비율로 넣어주기 위함
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((399744, 13), (399744,), (99936, 13), (99936,))

## 모델 생성 및 학습

In [139]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

## 예측 및 모델 평가

In [143]:
# 예측
y_hat = model.predict(x_test)

# 모델 평가
from sklearn.metrics import accuracy_score, confusion_matrix
print(f'분류 정확도: {accuracy_score(y_test, y_hat):.3f}')
print(confusion_matrix(y_test, y_hat))

분류 정확도: 1.000
[[86292     0]
 [    0 13644]]


## Feature Importance

In [146]:
# 도착 지연 여부 판단시 주요 파라미터가 무엇이었는지 확인하기

importance = { k:v for k, v in zip(x.columns, model.feature_importances_)}
df_importance = pd.DataFrame(pd.Series(importance), columns=['importance'])
df_importance = df_importance.sort_values('importance',  ascending=False)
df_importance
# 중요도 기준 내림차순으로 정렬
# 도착 지연 여부 판단시 가장 중요한 feature는 ArrDelay

,importance
ArrDelay,0.647003
DepDelay,0.178629
DepDel15,0.164814
DepTimeBlk,0.002308
Carrier,0.001931
ArrTimeBlk,0.001820
DayOfWeek,0.000891
Ori-state,0.000452
Dest-airport,0.000449
Ori-airport,0.000434
